# Hearing a Rectangle

In [1]:
# Imports
#
# C:\Users\sulta\AppData\Local\Programs\Python\Python310\Scripts\pip install scipy
#
import numpy as np
from scipy import stats, signal
from scipy.optimize import minimize
from scipy.optimize import curve_fit
from scipy.optimize import shgo

In [2]:
# Parameters of the "true" rectangle
a_true = 1.0
b_true = 3.0
pi = np.pi

# sollte locker für 1000 Eigenwerte reichen
m_max, n_max = 800, 800

ms = np.arange(1, m_max + 1)
ns = np.arange(1, n_max + 1)
M, N = np.meshgrid(ms, ns, indexing="ij")

# Dirichlet-Eigenwerte: λ_{m,n} = π²(m²/a² + n²/b²)
lambdas = pi**2 * (M**2 / a_true**2 + N**2 / b_true**2)

# Flatten & sort
l_flat = lambdas.ravel()
sort_idx = np.argsort(l_flat)
l_sorted = l_flat[sort_idx]

Nvals = 10000
eigvals = l_sorted[:Nvals]

print("Erste 12 Eigenwerte (gerundet):")
print([round(float(x), 4) for x in eigvals[:12]])

Erste 12 Eigenwerte (gerundet):
[10.9662, 14.2561, 19.7392, 27.4156, 37.2852, 40.575, 43.8649, 49.348, 49.348, 57.0244, 63.6041, 66.894]


In [3]:
def reconstruct_aspect_ratio_fourier_analysis(eigvals, N_total_fit=9500):
    """
    Rekonstruiert die Seitenlängen (a, b) und R = a/b mittels der Fourier-Analyse 
    der spektralen Fluktuationen (Geometrisch/Spektrale Dualität).
    
    Diese Methode ist index-frei und nutzt die Periodizität des Spektrums.
    
    Args:
        eigvals (list oder np.ndarray): Die sortierten Eigenwerte (λ_k).
        N_total_fit (int): Anzahl der Eigenwerte, die für die Analyse verwendet werden.
        
    Returns:
        tuple: (rekonstruiertes_a, rekonstruiertes_b, rekonstruiertes_R)
    """
   
    N_total = min(len(eigvals), N_total_fit)
    eigvals = np.array(eigvals[:N_total])
    k_vals = np.arange(1, N_total + 1)
    
    print("=========================================================================")
    print("Rekonstruktion der Seitenlängen (FINAL: SPEKTRALE FOURIER-ANALYSE)")
    print("=========================================================================")
    
    # ----------------------------------------------------
    ## 1. Schritt: Rekonstruktion der Fläche (A = ab)
    # ----------------------------------------------------
    start_index_A = int(0.3 * N_total)
    slope_A, _, _, _, _ = stats.linregress(eigvals[start_index_A:], k_vals[start_index_A:])
    A_hat = 4 * pi * slope_A
    print(f"--- Schritt 1: Fläche (Area) ---")
    print(f"Rekonstruierte Fläche A = a*b ≈ {A_hat:.4f} (Wahr: {a_true*b_true:.1f})")

    # ----------------------------------------------------
    ## 2. Schritt: Analyse der Spektralen Fluktuationen
    # ----------------------------------------------------
    print(f"\n--- Schritt 2: Fourier-Analyse (Bestimmung der Längen L=2a, 2b) ---")
    
    # Transformation zu t_k = sqrt(lambda_k) (Die Frequenzvariable)
    t_k = np.sqrt(eigvals)
    
    # Die Fluktuationsfunktion F(t_k) = k - N_Weyl(lambda_k)
    # Da der Goursat-Term L*sqrt(lambda) numerisch instabil war, verwenden wir nur 
    # den Flächentem (Weyl) und lassen den L-Term in der Fluktuation F(t_k).
    N_weyl = (A_hat / (4 * pi)) * eigvals
    F_k = k_vals - N_weyl
    
    # WICHTIG: Die Fouriertransformation der Fluktuationen F(t_k) liefert die Längen.
    # Wir führen die Analyse auf dem transformierten Spektrum t_k durch.
    
    # 1. Erstellen einer diskreten Frequenzachse für die DFT (Längenachse)
    # Max. Länge (L) sollte L_max < N_total / (2 * delta_t) sein.
    L_max = 10.0 # Wir erwarten L=2a=2 und L=2b=6 (oder L=6, L=2)
    N_fft = 2**12 # Feste FFT-Größe für hohe Auflösung

    # 2. Erstellen des Signals für die DFT (diskretisierte Fluktuationen)
    # Wir verwenden eine Sinc-Interpolation oder eine direkte Akkumulation
    
    # Robustere Methode: Direkte DFT-Akkumulation auf den Eigenwerten t_k
    # Spektraldichte S(L) = | Sum_{k} exp(-i * L * t_k) |^2
    
    L_axis = np.linspace(0.1, L_max, N_fft // 2) 
    spectral_density = np.zeros_like(L_axis)
    
    # Gewichtung (z.B. Gauß-Fenster) um Randeffekte zu reduzieren.
    weight = np.exp(-0.5 * (t_k / t_k[-1])**2) 
    
    # Berechnung der Spektraldichte
    for idx, L in enumerate(L_axis):
        # Akkumuliere die Fouriertransformation: Summe F_k * exp(-i L t_k)
        dft_sum = np.sum(F_k * np.exp(-1j * L * t_k) * weight)
        spectral_density[idx] = np.abs(dft_sum)**2

    # 3. Finde die dominanten Peaks in der Spektraldichte
    # Die Peaks entsprechen 2*a und 2*b.
    
    # Finde die beiden größten Peaks (ausgenommen sehr kleine L)
    peak_indices = signal.find_peaks(spectral_density, height=np.max(spectral_density) * 0.1, distance=50)[0]
    
    # Sortiere nach Peak-Höhe und wähle die zwei größten
    peak_heights = spectral_density[peak_indices]
    sorted_peaks = peak_indices[np.argsort(peak_heights)[::-1]]
    
    # Extrahiere die Längen (2a, 2b) aus den Peak-Positionen
    if len(sorted_peaks) < 2:
         print("Fehler: Konnte nicht genügend Peaks (Längen) finden.")
         return np.nan, np.nan, np.nan
        
    L1_hat = L_axis[sorted_peaks[0]]
    L2_hat = L_axis[sorted_peaks[1]]

    # Längen ordnen: 2*a < 2*b
    L_short, L_long = min(L1_hat, L2_hat), max(L1_hat, L2_hat)
    
    a_hat = L_short / 2.0
    b_hat = L_long / 2.0
    
    print(f"Gefundene Längen L = 2*a, 2*b ≈ ({L_short:.4f}, {L_long:.4f})")
    print(f"Rekonstruierte Seitenlängen (a, b) ≈ ({a_hat:.4f}, {b_hat:.4f})")
    
    # ----------------------------------------------------
    ## 3. Schritt: Endergebnis
    # ----------------------------------------------------
    
    # Rekonstruiertes R aus a_hat und b_hat
    R_hat = a_hat / b_hat

    print(f"Bester Fit für Seitenverhältnis R = a/b ≈ {R_hat:.4f}")
    
    print(f"\n--- Schritt 3: Endergebnis ---")
    print(f"Rekonstruierte Seitenlängen (geordnet): (a, b) ≈ ({a_hat:.4f}, {b_hat:.4f})")
    
    a_true_final, b_true_final = min(a_true, b_true), max(a_true, b_true)
    print(f"Wahre Seitenlängen (geordnet): (a, b) = ({a_true_final:.1f}, {b_true_final:.1f})")
    print(f"Abweichung: |a_final - a_true|: {abs(a_hat - a_true_final):.4f}")
    print(f"Abweichung: |b_final - b_true|: {abs(b_hat - b_true_final):.4f}")
    
    print("\n--- Rekonstruktion abgeschlossen ---")

    return a_hat, b_hat, R_hat

In [4]:
reconstruct_aspect_ratio_fourier_analysis(eigvals)

Rekonstruktion der Seitenlängen (FINAL: SPEKTRALE FOURIER-ANALYSE)
--- Schritt 1: Fläche (Area) ---
Rekonstruierte Fläche A = a*b ≈ 2.9747 (Wahr: 3.0)

--- Schritt 2: Fourier-Analyse (Bestimmung der Längen L=2a, 2b) ---
Gefundene Längen L = 2*a, 2*b ≈ (2.0055, 6.3244)
Rekonstruierte Seitenlängen (a, b) ≈ (1.0028, 3.1622)
Bester Fit für Seitenverhältnis R = a/b ≈ 0.3171

--- Schritt 3: Endergebnis ---
Rekonstruierte Seitenlängen (geordnet): (a, b) ≈ (1.0028, 3.1622)
Wahre Seitenlängen (geordnet): (a, b) = (1.0, 3.0)
Abweichung: |a_final - a_true|: 0.0028
Abweichung: |b_final - b_true|: 0.1622

--- Rekonstruktion abgeschlossen ---


(np.float64(1.0027601367855399),
 np.float64(3.1621885686370295),
 np.float64(0.3171095319017458))